In [9]:
import pandas as pd
import re
from nltk.corpus import stopwords
import pymorphy3
from tqdm.auto import tqdm

In [10]:
aviation_data = pd.read_csv('../data/raw/aviation.csv')
road_transport_data = pd.read_csv('../data/raw/road_transport.csv')

print(aviation_data.info())
print(road_transport_data.info())

/var/folders/bt/nbybvdjd4dl4j63pnps0226w0000gn/T/ipykernel_32604/36430761.py:1: DtypeWarning: Columns (21,22,24,26,28,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  aviation_data = pd.read_csv('../data/raw/aviation.csv')
/var/folders/bt/nbybvdjd4dl4j63pnps0226w0000gn/T/ipykernel_32604/36430761.py:2: DtypeWarning: Columns (9,22,24,26,28,29,30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  road_transport_data = pd.read_csv('../data/raw/road_transport.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151286 entries, 0 to 151285
Data columns (total 33 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Unnamed: 0            151286 non-null  int64  
 1   inner_type            151286 non-null  object 
 2   comments              151286 non-null  object 
 3   marked_as_ads         151286 non-null  int64  
 4   hash                  151285 non-null  object 
 5   type                  151286 non-null  object 
 6   push_subscription     143075 non-null  object 
 7   is_added_to_main_tab  151286 non-null  bool   
 8   attachments           151286 non-null  object 
 9   date                  151286 non-null  int64  
 10  from_id               151286 non-null  int64  
 11  id                    151286 non-null  int64  
 12  is_favorite           151286 non-null  bool   
 13  likes                 148953 non-null  object 
 14  owner_id              151286 non-null  int64  
 15  

In [11]:
data = pd.concat([aviation_data, road_transport_data])

data = data[data['marked_as_ads'] == 0]

data = data[['text', 'y']]

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 362370 entries, 0 to 213844
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    324894 non-null  object
 1   y       362370 non-null  object
dtypes: object(2)
memory usage: 8.3+ MB


In [12]:
data.dropna(inplace=True)
data.drop_duplicates(keep='first', inplace=True)

y_map = {
    'aviation': 0,
    'road_transport': 1
}
data['y'] = data['y'].map(y_map)

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 310580 entries, 0 to 213842
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    310580 non-null  object
 1   y       310580 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 7.1+ MB


In [13]:
morph = pymorphy3.MorphAnalyzer()
stopwords = set(stopwords.words('russian'))

garbage_list = [
    'http', 'https', 'vk', 'cc', 'com', 'ru', 'me', 'avtoobmen'
]
stopwords.update(garbage_list)

def preprocess_text(text):
    if not isinstance(text, str): 
        return ""

    text = text.lower()
    text = re.sub(r'(https?://\S+)|(www\.\S+)|(\w+\.(com|net|org|ru|me|cc|to)/\S+)', ' ', text)
    text = re.sub(r'\[(id|club)\d+\|.*?\]', '', text)
    text = re.sub(r'[^a-zа-яё\s]', ' ', text)

    words = text.split()

    lemmas = []
    for word in words:
        if len(word) > 1:
            lemma = morph.parse(word)[0].normal_form
            
            if lemma not in stopwords:
                lemmas.append(lemma)

    return " ".join(lemmas)


print("\nНачало предобработки текста...")

tqdm.pandas()
data['processed_text'] = data['text'].progress_apply(preprocess_text)

print("Предобработка текста завершена.")

display(data.head())


Начало предобработки текста...


100%|██████████| 310580/310580 [25:21<00:00, 204.11it/s] 


Предобработка текста завершена.


,text,y,processed_text
0,У нас сегодня двойной праздник! Отмечаем 10-ле...,0,сегодня двойной праздник отмечать летие аэрофл...
1,"❄ Хоть уже зима, сегодняшний день выдался очен...",0,зима сегодняшний день выдаться очень уютный пр...
2,Что делаете на Новый год?\n\nКак насчёт смены ...,0,делать новый год насчёт смена привычный обстан...
3,"Для тех, кто часто летает, выпустили Бизнес-пр...",0,часто летать выпустить бизнес проездной абонем...
4,Декабрь — месяц больших покупок. Превращайте н...,0,декабрь месяц больший покупка превращать новог...


In [14]:
data.to_csv('../data/processed/processed_data.csv', index=False)

In [15]:
print(*data['processed_text'][0])

print('-' * 150)

print(*data['text'][0])

сегодня двойной праздник отмечать летие аэрофлот техникс день гражданский авиация случай совпадение случайный крупный провайдер технический обслуживание ремонт воздушный судно территория россия снг эксперт авиация невозможный полёт профессионал любой высота плечо коллега поздравлять юбилей гордиться совместный достижение ускорить публикация написать пометка хотеть пост очередь телеграмма барахолка авто москва канал барахолка авто москва диалог
------------------------------------------------------------------------------------------------------------------------------------------------------
У нас сегодня двойной праздник! Отмечаем 10-летие «Аэрофлот Техникс» в День гражданской авиации. Тот случай, когда совпадения не случайны. 

Крупнейший провайдер технического обслуживания и ремонта воздушных судов на территории России и СНГ. Без этих экспертов авиации невозможен ни один полёт. С профессионалами любые высоты по плечу!

Коллеги, поздравляем с юбилеем! Гордимся совместными достижениям